# RAG application built on gemini

In [6]:
!pip install langchain
!pip install langchain_community
!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 4.5 MB/s eta 0:00:00


ValueError: File path yolov9_paper.pdf is not a valid file or url

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
loader = PyPDFLoader("/content/drive/MyDrive/gemini_rag_demo/yolov9_paper.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [9]:
len(data)

18

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  96


In [11]:
docs[7]

Document(metadata={'source': '/content/drive/MyDrive/gemini_rag_demo/yolov9_paper.pdf', 'page': 1}, page_content='ditional layers to combine repeatedly fed input data, which\nwill significantly increase the inference cost. In addition,\nsince the input data layer to the output layer cannot have a\ntoo deep path, this limitation will make it difficult to model\nhigh-order semantic information during the training pro-\ncess. As for masked modeling, its reconstruction loss some-\ntimes conflicts with the target loss. In addition, most mask\nmechanisms also produce incorrect associations with data.\nFor the deep supervision mechanism, it will produce error\naccumulation, and if the shallow supervision loses informa-\ntion during the training process, the subsequent layers will\nnot be able to retrieve the required information. The above\nphenomenon will be more significant on difficult tasks and\nsmall models.\nTo address the above-mentioned issues, we propose a\nnew concept, which is prog

In [12]:
!pip install langchain_chroma
!pip install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00


In [15]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
#from dotenv import load_dotenv
#load_dotenv()
os.environ["GOOGLE_API_KEY"]='key'

#Get an API key:
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [16]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [17]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")


In [18]:
len(retrieved_docs)

10

In [19]:
print(retrieved_docs[5].page_content)

target task, it also overcomes the problems encountered by
mask modeling. The proposed PGI mechanism can be ap-
plied to deep neural networks of various sizes and is more
general than the deep supervision mechanism, which is only
suitable for very deep neural networks.
In this paper, we also designed generalized ELAN
(GELAN) based on ELAN [65], the design of GELAN si-
multaneously takes into account the number of parameters,
computational complexity, accuracy and inference speed.
This design allows users to arbitrarily choose appropriate
computational blocks for different inference devices. We
combined the proposed PGI and GELAN, and then de-
signed a new generation of YOLO series object detection
system, which we call YOLOv9. We used the MS COCO
dataset to conduct experiments, and the experimental results
verified that our proposed YOLOv9 achieved the top perfor-
mance in all comparisons.
We summarize the contributions of this paper as follows:


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [27]:
response = rag_chain.invoke({"input": "summarize what this pdf is about"})
print(response["answer"])

This research paper proposes a new deep learning method called YOLOv9, which addresses the information loss that occurs in traditional deep neural networks during the feedforward process. 

YOLOv9 introduces two novel components: Programmable Gradient Information (PGI) and Generalized ELAN (GELAN). PGI utilizes reversible architectures and multi-level auxiliary information to preserve input data and provide more reliable gradients for model training. GELAN, a new network architecture, combines the strengths of CSPNet and ELAN for improved performance. 

